# The Bin Packing problem 

The [bin packing problem](https://en.wikipedia.org/wiki/Bin_packing_problem) is an optimization problem where given a number of items with an assigned weight, we look at the best way to group the items minimizing the number of bins or containers needed to store them. The restriction, in this case, is the capacity of the bins which cannot surpass a certain weight. This problem has many real applications in areas such as loading trucks with a weight restriction, filling up containers, and FPGA semiconductors chip design. 

In terms of complexity, the bin packing problem is an NP-hard problem. However, there are efficient algorithms that allow the arrangement of a large number of items. One of them is the first fit, which provides a fast but not optimal solution to the problem. 

For our problem, we will explore the solution to the bin packing problem, using a quantum computing representation in terms of quadratic unconstraint binary optimization (QUBO) and using the quantum approximation optimization (QAOA) algorithm. 

### Problem statement

minimize $$K = \sum_{j=1}^m y_j$$

subject to:

$$\sum_{i=1}^n s(i) x_{ij} \le B y_j \qquad  \forall \ j=1,...,m$$
$$\sum_{j=1}^m x_{ij} = 1  \qquad \forall \ i = 1, ..., n$$
$$x_{ij}\in  \{0,1\} \qquad \forall \ i=1,..,n \qquad j=1,..,m$$
$$y_{j}\in  \{0,1\} \qquad \forall \ j=1,..,m $$

- n is the number of items
- m is the number of bins
- $s(i)$ is the i-th item weight
- B is the maximum weight of the bin
- $x_{ij}$ is the variable that represent if the item i is in the bin j.
- $y_j$ is the variable that represent if bin j is used

In [1]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa.workflows.optimizer import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel
from openqaoa.devices import create_device

# method to find the corrects states for the QAOA boject 
from openqaoa.utilities import ground_state_hamiltonian

## Generate the data

It generates the data necessary to create the BinPacking problem, i.e. to consider number of items, macimum number of bins, max weight of a bin, and the weight of each item.

In [2]:
# For the case of reproducibility this seed is configured
np.random.seed(1)

# Number of items
num_items = 2 

# Limit the maximum number of bins
num_bins = num_items

# Limit the max weight of a bin
max_weight = 7 

# Randomly picking the item weight
weights = np.random.randint(1, max_weight, num_items)

## Obtain the Quadratic problem from DOCPLEX

Once it is obtained the values for the binpacking problem, the next step is to translate it into a docplex model. Considering the above data the docplex model to solve it is given by 

In [3]:
# Construct model using docplex starting with the name of the Model object
mdl = Model("BinPacking")

# List of binary variables that represent the bins
y = mdl.binary_var_list(num_bins, name="y")

# List of binary variables that represent the items on the specific bin
x =  mdl.binary_var_matrix(num_items, num_bins, "x") 

# Design the objective function
objective = mdl.sum(y)

# This problem is minize the objective function
mdl.minimize(objective)

# Indicate the equality constraints
for i in range(num_items):
    # First set of constraints: the items must be in any bin
    mdl.add_constraint(mdl.sum(x[i, j] for j in range(num_bins)) == 1)

# Indicate the inequality constraints
for j in range(num_bins):
    # Second set of constraints: weight constraints
    mdl.add_constraint(mdl.sum(weights[i] * x[i, j] for i in range(num_items)) <= max_weight * y[j])

# Print a summary of the Docplex model
mdl.prettyprint()

// This file has been generated by DOcplex
// model name is: BinPacking
// var contrainer section
dvar bool y[2];
dvar bool x[2][2];

minimize
 y_0 + y_1;
 
subject to {
 x_0_0 + x_0_1 == 1;
 x_1_0 + x_1_1 == 1;
 6 x_0_0 + 4 x_1_0 <= 7 y_0;
 6 x_0_1 + 4 x_1_1 <= 7 y_1;

}


## Solving the problem using QAOA


The class `FromDocplex2IsingModel` from OpenQAOA converts the docplex representation of the problem to its QUBO representation in Ising encoding (-1, 1). From there, it is only required setting the QAOA model and solve the QUBO.

In [4]:
# Converting the Docplex model of binpacking into its qubo representation
qubo_bin = FromDocplex2IsingModel(mdl) 

# Ising encoding of the QUBO problem for binpacking problem
ising_encoding_bin = qubo_bin.ising_model 

# Docplex encoding of the QUBO problem for binpacking problem
mdl_qubo_docplex_bin = qubo_bin.qubo_docplex
mdl_qubo_docplex_bin.prettyprint()

// This file has been generated by DOcplex
// model name is: Copy of Copy of BinPacking
// var contrainer section
dvar bool y[2];
dvar bool x[2][2];
dvar bool slack_C2[3];
dvar bool slack_C3[3];

// single vars section
dvar bool y_0;
dvar bool y_1;
dvar bool x_0_0;
dvar bool x_0_1;
dvar bool x_1_0;
dvar bool x_1_1;
dvar bool slack_C2_0;
dvar bool slack_C2_1;
dvar bool slack_C2_2;
dvar bool slack_C3_0;
dvar bool slack_C3_1;
dvar bool slack_C3_2;

minimize
 y_0 + y_1 - 6 x_0_0 - 6 x_0_1 - 6 x_1_0 - 6 x_1_1 [ 147 y_0^2 - 252 y_0*x_0_0
 - 168 y_0*x_1_0 - 42 y_0*slack_C2_0 - 84 y_0*slack_C2_1 - 168 y_0*slack_C2_2
 + 147 y_1^2 - 252 y_1*x_0_1 - 168 y_1*x_1_1 - 42 y_1*slack_C3_0
 - 84 y_1*slack_C3_1 - 168 y_1*slack_C3_2 + 111 x_0_0^2 + 6 x_0_0*x_0_1
 + 144 x_0_0*x_1_0 + 36 x_0_0*slack_C2_0 + 72 x_0_0*slack_C2_1
 + 144 x_0_0*slack_C2_2 + 111 x_0_1^2 + 144 x_0_1*x_1_1 + 36 x_0_1*slack_C3_0
 + 72 x_0_1*slack_C3_1 + 144 x_0_1*slack_C3_2 + 51 x_1_0^2 + 6 x_1_0*x_1_1
 + 24 x_1_0*slack_C2_0 + 48 x_1

Using the pyquil backend and 1024 shots with a p value equals to 3, with  betas and gammas  of $0.01 * \pi$ 

In [5]:
# Indicate the device, this case is a local simulator
device = create_device("local", 'pyquil.statevector_simulator')

# Initilize the QAOA object
qaoa_bin = QAOA(device)

# Set the parameters to work the QAOA algorithm
qaoa_bin.set_backend_properties(n_shots=1024, seed_simulator=1)
rep = 3
qaoa_bin.set_circuit_properties(p=rep, init_type="custom", variational_params_dict={"betas":rep*[0.01*np.pi],"gammas":rep*[0.01*np.pi]})
qaoa_bin.compile(ising_encoding_bin)

# Run the QAOA algorithm
qaoa_bin.optimize()

show the best 5 states for this binpacking problem

In [6]:
qaoa_bin_dict = qaoa_bin.results.lowest_cost_bitstrings(5)
pd.DataFrame(qaoa_bin_dict)

,solutions_bitstrings,bitstrings_energies,probabilities
0,111001100110,2.0,6.772599e-07
1,110110110100,2.0,6.772599e-07
2,100010110000,4.0,2.518399e-06
3,010100000100,4.0,1.779673e-05
4,101000100000,4.0,1.779673e-05


Chek the correct answer using the ground_state_hamiltonian and the 2 correct answer are in the best 5 best states.

In [7]:
# Check the exactly solution
sol = ground_state_hamiltonian(qaoa_bin.cost_hamil)
sol

(2.0, ['110110110100', '111001100110'])

### 2.2 Solution using DOCPLEX

The QUBO problem can be solved classically using DOCPLEX. This is a good comparision between QAOA against classical optimizers.

**Note: For the next cell you  will need to install cplex with the command** `pip install cplex>=22.1.0.0` 

In [8]:
# docplex QUBO model
mdl_qubo_bin = qubo_bin.qubo_docplex 

# Obtain the docplex solution
sol_bin_docplex = mdl_qubo_bin.solve()
sol_bin = {i.name: int(sol_bin_docplex.get_value(i)) for i in mdl_qubo_bin.iter_binary_vars()}
mdl_qubo_bin.print_solution(print_zeros=True)

objective: 2.000
  y_0=1
  y_1=1
  x_0_0=0
  x_0_1=1
  x_1_0=1
  x_1_1=0
  slack_C2_0=1
  slack_C2_1=1
  slack_C2_2=0
  slack_C3_0=1
  slack_C3_1=0
  slack_C3_2=0


The solution is `110110110100`,  being one state as in the quantum algorithm